In [1]:
# %matplotlib notebook
%matplotlib widget
%load_ext autoreload
%autoreload 2

In /Users/vxh710/.matplotlib/stylelib/paper.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /Users/vxh710/.matplotlib/stylelib/presentation.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /Users/vxh710/.matplotlib/stylelib/poster_dark.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /Users/vxh710/.matplotlib/stylelib/paper_twocol.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /Users/vxh710/.matplotlib/stylelib/poster.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /Users/vxh710/.matplotlib/stylelib/presentation_dark.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import fits

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/Users/vxh710/PhD/software/elle/elle")
import utils
# from utils.io import read

# 1. Reading the CCFs

We will start by loading in the cross-correlation functions (CCFs). This step will be similar for data from e.g. HARPS, HARPS-N, SOPHIE, CORALIE, and ESPRESSO since their pipelines compute the CCFs. For other spectrographs where this may not the case, a CCF will first have to be computed. This is outside the scope of this tutorial.

Once the data is read we will do a number of adjustments. 

*i)* The velocity step size in the HARPS data product we are using is 0.25 km/s, which is *oversampled*. We will only keep every one out of four points, which will more closely resemble the natural resolution of HARPS. 

*ii)* For this particular data the CCF covers a wide range of the continuum. The continuum is not as important to us as the line core itself, so we are going to mask out most of the continuum with a mask `m`, which will simplify things later.

*iii)* One can also notice that the velocity grids (the first reference to `rv` below) from all the observations line up, i.e. have the same reference value and step size. For now, we are going to assume that the radial velocity grids are aligned (as in this case), which means we do not have to store the velocity array for each individual CCF. In this case, we let `rv` be the common grid for all the CCFs.

**Note**: The velocity grids are not necessarily always aligned, depending on the data reduction. If they are not aligned, it will simplify things later on if we first align them. If only the reference value is different for some CCFs but they have the same step size and most of the grids align, then the simplest solution is to discard the data that are outside of the common range for all of them. If most of the grid does not align, a solution may be to resample the CCFs (see later), or let `rv` be a list for the remainder of the tutorial so that each CCF has its own unique velocity grid. 



In [3]:
data_dir = "/Users/vxh710/PhD/projects/reloaded/data/HD189733/fits/2006-09-08"

# save the full path to each FITS file in a list (exclude .DS_Store files on MacOS)
files = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if not x.startswith('.')]

files.sort() # files read are not necessarily in order, so order by timestamp in filename

In [29]:
# from astropy.io import fits

# f = fits.open('/Users/vxh710/PhD/projects/reloaded/data/HD189733/fits/2006-09-08/HARPS.2006-09-08T01:19:45.782_ccf_K5_A.fits')
# f[0].header

# HIERARCH ESO DRS CCF RVC = -2.16965003456311 / Baryc RV (drift corrected) (km/s)
# HIERARCH ESO DRS CCF CONTRAST = 39.9438996989951 / Contrast of  CCF (%)         
# HIERARCH ESO DRS CCF FWHM = 7.52428638475369 / FWHM of CCF (km/s)               
# HIERARCH ESO DRS CCF RV = -2.16965003456311
# HIERARCH ESO DRS BJD
# EXPTIME

In [32]:
rv, ccf, obs = utils.io.read(files, oversample=4)

rv = np.median(rv, axis=0)

In [33]:
rv

array([-252., -251., -250., -249., -248., -247., -246., -245., -244.,
       -243., -242., -241., -240., -239., -238., -237., -236., -235.,
       -234., -233., -232., -231., -230., -229., -228., -227., -226.,
       -225., -224., -223., -222., -221., -220., -219., -218., -217.,
       -216., -215., -214., -213., -212., -211., -210., -209., -208.,
       -207., -206., -205., -204., -203., -202., -201., -200., -199.,
       -198., -197., -196., -195., -194., -193., -192., -191., -190.,
       -189., -188., -187., -186., -185., -184., -183., -182., -181.,
       -180., -179., -178., -177., -176., -175., -174., -173., -172.,
       -171., -170., -169., -168., -167., -166., -165., -164., -163.,
       -162., -161., -160., -159., -158., -157., -156., -155., -154.,
       -153., -152., -151., -150., -149., -148., -147., -146., -145.,
       -144., -143., -142., -141., -140., -139., -138., -137., -136.,
       -135., -134., -133., -132., -131., -130., -129., -128., -127.,
       -126., -125.,

In [35]:
m = (rv < (np.median(obs['rv']) - 1.5 * np.median(obs['fwhm']))) | (rv > (np.median(obs['rv']) + 1.5 * np.median(obs['fwhm'])))

fig, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].plot(rv, ccf[0])
ax[0].plot(rv[m], ccf[0][m], '.C1', label='continuum')
ax[0].set_xlabel('velocity (km/s)')
ax[0].set_ylabel('ccf flux')
ax[0].set_title('not normalized')
ax[0].legend()

# normalize by continuum
ccf /= np.median(ccf[:,m], axis=1)[:,None] # estimate continuum

ax[1].plot(rv, ccf[0])
ax[1].set_xlabel('velocity (km/s)')
ax[1].set_ylabel('ccf flux')
ax[1].set_title('normalized by continuum');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# the timestamps of the observations
time = np.array([fits.getval(x, 'HIERARCH ESO DRS BJD') for x in files])

# the CCFs are given per order, and we are interested in the last element which is the combined CCF from all orders
ccf = [fits.getdata(x)[-1,:] for x in files]

# next, we create the radial velocity grid - Note: the headers may differ between instruments
start = [fits.getval(x, 'CRVAL1') for x in files] # the velocity at the first (reference) pixel
step = [fits.getval(x, 'CDELT1') for x in files] # the velocity step size
rv = [np.arange(start[i], start[i] + len(ccf[i]) * step[i], step[i]) for i in range(len(ccf))]

# if velocity grids are the same for all CCFs we use a common velocity grid
# NOTE: verify that this is true for your data
rv = rv[0] 

# create mask to remove most of continuum
rv_min = rv[np.argmin(ccf[0])] # approximate velocity of CCF line centre
m = (rv > (rv_min - 40)) & (rv < (rv_min + 40)) # can use same mask since RV doesn't change much during sequence

# discard oversampled points
rv = rv[m][::4]
ccf = np.atleast_2d(ccf)[:,m][:,::4] # note that the velocity grids should be the same for all CCFs in order to cast the CCF list into a 2d array

# normalize the CCFs by continuum using the computed line centres and widths
rvc = np.array([fits.getval(x, 'HIERARCH ESO DRS CCF RVC') for x in files])
rvc_err = np.array([fits.getval(x, 'HIERARCH ESO DRS CCF NOISE') for x in files])
# line_centre = np.median([fits.getval(x, 'HIERARCH ESO DRS CCF RVC') for x in files])
line_width = np.median([fits.getval(x, 'HIERARCH ESO DRS CCF FWHM') for x in files])

# select continuum by some distance from line centre
# Note: variable name has no relation to previous mask, which we also called `m`
m = (rv < (np.median(rvc) - 1.5 * line_width)) | (rv > (np.median(rvc) + 1.5 * line_width))

fig, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].plot(rv, ccf[0])
ax[0].plot(rv[m], ccf[0][m], '.C1', label='continuum')
ax[0].set_xlabel('velocity (km/s)')
ax[0].set_ylabel('ccf flux')
ax[0].set_title('not normalized')
ax[0].legend()

# normalize by continuum
ccf /= np.median(ccf[:,m], axis=1)[:,None] # estimate continuum

ax[1].plot(rv, ccf[0])
ax[1].set_xlabel('velocity (km/s)')
ax[1].set_ylabel('ccf flux')
ax[1].set_title('normalized by continuum');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:



# rv_min = rv[np.argmin(ccf[0])] # approximate velocity of CCF line centre
# m = (rv > (rv_min - 40)) & (rv < (rv_min + 40))

# ncols = 3
# nrows = int(len(ccf) / ncols) + 1

# fig, axes = plt.subplots(nrows, ncols, figsize=(10, nrows*3))
# axes = axes.flatten()

# for i in range(len(ccf)):
#     axes[i].plot(rv[m], ccf[i][m])



# 2. Light curve normalization and Keplerian correction

The CCF flux is not normalized by reference stars, so we will have to normalize them manually to account for the loss of light during transit. We will also have to resample the CCFs onto a new grid to correct for the Keplerian slope of the star due to the planet at the time of observations.

For this, we will need to compute a transit model and radial velocity curve using some pre-existing software package. In this tutorial we will use `exoplanet`, but really any software that can compute transit light curves and radial velocity curves will do. 

First, we will define some planetary and orbital parameters for the light curve and radial velocity models, and calculate the phases and duration of the transit which will be important later.

In [5]:
import sys
sys.path.append("/Users/vxh710/PhD/software/elle")
from utils import

# dur    = 1.827/24                       # duration between first and fourth transit contact (days)
ror    = 0.15667                        # planet-star radius ratio
period = 2.21857567                     # orbital period (days)
t0     = 2454279.436714                 # transit time (BJD)
K      = 200.56                         # RV semi-amplitude (m/s)
incl   = 85.710                         # orbital inclination (deg)
aor    = 8.863                          # scaled separation
roa    = 1/aor
b      = np.cos(np.deg2rad(incl)) * aor # impact parameter

u      = np.array([0.816, 0])           # limb darkening coefficients for quadratic law


# get orbital phase of observations
phase = (time - t0) % period / period
phase[phase > 0.5] -= 1


# compute transit duration between contacts
dur14 = utils.get_14_transit_duration(period, 1/aor, ror, b, np.deg2rad(incl))
dur23 = utils.get_23_transit_duration(period, 1/aor, ror, b, np.deg2rad(incl))

# create masks to easily select points in/out of transit
m14 = utils.get_transit_mask(phase, dur14/period)
m23 = utils.get_transit_mask(phase, dur23/period)

Next, we compute the transit light curve and radial velocity (Keplerian slope only, *without* Rossiter-McLaughlin model). Here we use `exoplanet`, but any other code will do. What we're interested in are the light curve and radial velocity slope at the times of our observations, so if you can compute that using another code and save those in arrays called `lcmod` and `rvmod`, then the rest of the tutorial should still work.

In [6]:
import exoplanet as xo

# the `duration` parameter in `exoplanet` refers to the interval between the halfway points of ingress and egress,
# i.e. contact times 1.5 and 3.5
dur_xo = dur14 - 0.5 * (dur14 - dur23)
# dur_xo = period / (np.pi * aor) * np.sqrt(1 - b**2) # Winn 2010, Transits and Occultations

# exposure time of RV observations to correct for finite integration time
texp = np.array([fits.getval(x, 'EXPTIME') for x in files]) / 60 / 60 / 24 

orbit = xo.orbits.KeplerianOrbit(
           duration=dur_xo,
           period=period,
           t0=t0,
           b=b
        )

def get_lc(x):
    return np.sum(
                (
                xo.LimbDarkLightCurve(u).get_light_curve(
                    orbit=orbit, r=ror, t=x, texp=np.median(texp)/60/60/24
                    )
                ).eval(), axis=-1) + 1

def get_rv(x):
    return orbit.get_radial_velocity(x, K=K).eval() / 1e3


# model at timestamps: compute these using another code if `exoplanet` is not installed on your system
lcmod = get_lc(time)
rvmod = get_rv(time)

# finer models for visualization
time_f  = np.linspace(time[0], time[-1], 200)
phase_f = np.linspace(phase.min(), phase.max(), 200)
lcmod_f = get_lc(time_f)
rvmod_f = get_rv(time_f)

fig, axes = plt.subplots(3,1, sharex=True, figsize=(6,6), gridspec_kw={"hspace":0.03})

# estimate systemic velocity from out-of-transit observations for visualization.
# this will fail if there are no data outside transit
vsys = np.average((rvc - rvmod)[~m14], weights=rvc_err[~m14])

axes[0].plot(phase_f*period, rvmod_f*1e3, label='model')
# axes[0].plot(phase*period, rvmod, '.', label="timestamps")
axes[0].errorbar(phase*period, (rvc - vsys)*1e3, yerr=rvc_err*1e3, capsize=0, fmt='.', label="data")
axes[0].set_ylabel('RV (km/s)')

axes[1].plot(phase_f*period, lcmod_f, label='model')
axes[1].plot(phase*period, lcmod, '.', label="timestamps")
axes[1].set_ylabel('normalised flux')


axes[2].errorbar(phase*period, (rvc - rvmod - vsys)*1e3, yerr=rvc_err*1e3, capsize=0, fmt='.', label="RM data\nwithout slope")
axes[2].set_ylabel('RV (m/s)')
axes[2].set_xlabel('phase (d)')

for ax in axes:
    ax.axvline(-0.5 * dur14, c="#aaaaaa")
    ax.axvline(0.5 * dur14, c="#aaaaaa")
    ax.axvline(-0.5 * dur23, ls='dashed', c="#aaaaaa")
    ax.axvline(0.5 * dur23, ls='dashed', c="#aaaaaa")
    ax.axvline(0, ls='dotted', c='#aaaaaa')
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Looks good! We can see that our model for the Keplerian slope has been removed from the bottom panel. Now that we know our model is good, we can correct the CCFs for the slope by resampling them onto a new grid. We will here also normalize our CCFs by the transit model.

In [7]:
ccf_lcnorm = utils.normalise_ccf(ccf, lcmod)#, err=ccf_err)
ccf_lcnorm += (1 - lcmod[:,None])

# we remove the first and last pixels since the interpolation may introduce weird spikes in the CCF
ccf_resampled = utils.resample2(np.tile(rv, (len(ccf_lcnorm), 1)), ccf_lcnorm, rvmod)[:,1:-1]

# 3. Create the master out of transit reference and residual CCFs

The reloaded Rossiter-McLaughlin approach aims at isolating the distortion of the CCF due to the planet transit. In order to do this, we will need to create a high SNR CCF profile based on our out of transit data, `master_ccf_out`. Because the CCFs have been resampled to correct for the Keplerian motion, they are in the system's barycentric reference frame so that their line centres are aligned.

In [8]:
# create master ccf by averaging out of transit observations
master_ccf_out = np.mean(ccf_resampled[~m14], axis=0)


plt.figure()
plt.plot(rv[1:-1], master_ccf_out, lw=1, c='k')
for i in range(len(ccf_resampled)):
    plt.plot(rv[1:-1], ccf_resampled[i], lw=0.5)
plt.xlabel('velocity (km/s)')
plt.ylabel('flux')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'flux')

As we see from the plot, all the CCFs are still centered on the systemic velocity, although the Doppler reflex motion due to the planet has been removed. To correct for this, we compute the line centre of `master_ccf_out` from a least-squares fit and correct our velocity grid for this to bring it to the stellar rest frame.

In [9]:
ccf_residual = master_ccf_out - ccf_resampled

# we estimate the photon noise on each CCF by calculating the spread in flux at each pixel for the out of transit CCFs
ccf_err = np.std(ccf_residual[~m14], axis=0)

# the error on the master CCF will be reduced by sqrt(n)
master_ccf_out_err = ccf_err / np.sqrt(np.count_nonzero(~m14))

# fit Gaussian to master out of transit CCF with least-squares minimization
popt, perr = utils.fit_ccf(rv[1:-1], master_ccf_out, yerr=master_ccf_out_err)

print(f"radial velocity of line centre is {popt[2]*1e3:.2f} +- {perr[2]*1e3:.2f} m/s")

# remove systemic velocity to bring CCFs into the stellar rest frame
master_rv = rv[1:-1] - popt[2]

radial velocity of line centre is -2226.88 +- 0.20 m/s


Here we have successfully isolated the planet shadow, where `ccf_residual` represents the light from the regions on the star that are occulted by the planet due to the transit. The line centres `ccf_residual` represent the surface radial velocity on the star at each transit epoch. We can plot these profiles and their characteristic *trace* below, showing that the regions on the star that the planet occulted is moving from negative to positive surface velocities, indicative of a prograde orbit.

In [10]:
fig = utils.plot_trace(phase, master_rv, ccf_residual, m14,
                       duration_14=dur14/period, # these arguments are supplied so that we can draw lines at the expected ingress/egress
                       duration_23=dur23/period,
                       period=period,
                       show_legend=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 4. Retrieving local stellar surface velocities

We will now retrieve the surface radial velocities on the star. In this step we will assume that the residual CCFs have a Gaussian shape, as the asymmetry due to convective blueshift has been subtracted off using `master_ccf_out`. If your residual line profiles are not simple Gaussians, then you can here also experiment with other functional forms (e.g. double Gaussian for M dwarfs, Bourrier et al. 2018). What is important is that all residual CCFs should be fitted with the same model.

In this tutorial we will use a simple least-squares fit to the residual line profiles because this is the fastest, however `elle` also has support for MCMC using `pymc3` for more in-depth analysis of any correlation between parameters.

In [38]:
popt, perr = utils.fit_ccf_residuals(master_rv, ccf_residual[m14], err=ccf_err)

In [39]:
ncols = 3
nrows = int(len(ccf_residual[m14]) / ncols) + 1

fig, axes = plt.subplots(nrows, ncols, figsize=(8, nrows*3))
axes = axes.flatten()

x = np.linspace(master_rv.min(), master_rv.max(), 200)
unit = 1e3 # ppt
# for i in range(len(ccf_residual[m14])):
for i in range(len(axes)):
    if i >= len(ccf_residual[m14]):
        axes[i].axis('off')
        continue
    axes[i].plot(x, utils.inverted_normal_distribution(x, *popt[i])*unit)
    axes[i].errorbar(master_rv, ccf_residual[m14][i]*unit, yerr=ccf_err*unit, capsize=0, fmt='none', c='k')
    
    axes[i].set_xlabel('RV (km/s)')
    axes[i].set_ylabel('flux (ppt)')
  
fig.tight_layout()
# models = np.atleast_2d([utils.inverted_normal_distribution(master_rv, *popt[i])
#                         for i in range(len(popt))])
# # print(models)
# fig = utils.plot_ccfs(np.ones_like(ccf_residual[m23]) * master_rv,
#                 ccf_residual[m23],
#                 ccf_err=np.ones_like(ccf_residual[m23]) * ccf_err,
#                      model=models);
# axes = fig.axes
# for i in range(len(axes)):
#     axes[i].plot(x, utils.inverted_normal_distribution(x, *popt[i]))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
_, depth, centre, width = popt.T[:,1:-1]
_, depth_err, centre_err, width_err = perr.T[:,1:-1]
phase_in = phase[m14][1:-1]

[2.618615   2.70307078 2.77112235 2.67580732 2.66974917 2.69457073
 2.76706525 2.71161125 2.49857269]


In [46]:
fig, axes = plt.subplots(3, 1, figsize=(6,6), gridspec_kw={'hspace':0.02})

fwhm = 2 * np.sqrt(2 * np.log(2)) * width
fwhm_err = 2 * np.sqrt(2 * np.log(2)) * width_err

axes[0].errorbar(phase_in, centre, yerr=centre_err, capsize=0, fmt='.')
axes[0].tick_params(labelbottom=False)
axes[0].set_ylabel('local RV (km/s)')

axes[1].errorbar(phase_in, depth*unit, yerr=depth_err*unit, capsize=0, fmt='.')
axes[1].tick_params(labelbottom=False)
axes[1].set_ylabel('local contrast (ppt)')

axes[2].errorbar(phase_in, fwhm, yerr=fwhm_err, capsize=0, fmt='.')
axes[2].set_xlabel('phase')
axes[2].set_ylabel('local FWHM (km/s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[6.16636709 6.36524526 6.52549445 6.30104472 6.28677885 6.34522918
 6.51594071 6.38535652 5.88368906]


Text(0, 0.5, 'local FWHM (km/s)')

In [43]:
from multiprocessing import Pool
import emcee
sys.path.append("/Users/vxh710/PhD/software/elle")
import model


threads = 2
walkers = 100
steps = 10000

istar = 90. # stellar inclination, i.e. fitting for vsini, and not v_eq
alpha = 0. # assume rigid body, no differential rotation

bounds = (-20, 20)

reloaded_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}

relo = model.ReloadedModel(phase_in, **reloaded_kwargs)

def _log_prior(theta):
        
    vs, vc = theta
    
    if vs < bounds[0] or vs > bounds[1]:
        return -np.inf
    elif vc < bounds[0] or vc > bounds[1]:
        return -np.inf

    return 0

def _log_likelihood(data, model, error):
        inv_sigma2 = 1/error**2
        return -0.5 * np.sum((data - model)**2 * inv_sigma2 - np.log(inv_sigma2))

def _log_probability(theta):

    # calculate prior and check the new parameters are within bounds
    l = _log_prior(theta)
    
    if not np.isfinite(l):
        return -np.inf
    
    # calculate vsini and lambda from the free parameters
    vs, vc = theta # vs = sqrt(vsini) * np.sin(lambda); vc = sqrt(vsini) * np.cos(lambda)
    vsini = vs**2 + vc**2  
    ell = np.rad2deg(np.arctan2(vs, vc))

    mod = relo(vsini, ell, istar, alpha) # calculate surface RV model

    l += _log_likelihood(centre, mod, centre_err)
    
    return l

parameters = ['vs', 'vc']
ndim = len(parameters)

init = []
for i in range(walkers):
    pos = np.random.uniform(*bounds, 2)
    
    # check that the initial positions are within prior bounds to be safe
    while not np.isfinite(_log_prior(pos)):
        pos = np.random.uniform(*bounds, 2)

    init.append(pos)

    
if threads > 1:
    os.environ["OMP_NUM_THREADS"] = "1"
    with Pool(processes=threads) as pool:
        sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability,
                                        pool=pool)
#                                         args=(phase, rv, rv_err))
        sampler.run_mcmc(init, steps, progress=True)
else:
    sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability)
#                                         args=(phase, rv, rv_err))
    sampler.run_mcmc(init, steps, progress=True)

100%|██████████| 10000/10000 [06:18<00:00, 26.43it/s]


In [228]:
# # utils.run_mcmc()

# # elle.run_mcmc(x, y, yerr, model_kwargs, )

# from multiprocessing import Pool
# import emcee
# sys.path.append("/Users/vxh710/PhD/software/elle")
# import model


# threads = 2
# walkers = 200
# steps = 5000


# istar = 90.
# alpha = 0.

# # roa_err = 0.020
# roa_err = 0.00025
# incl_err = 0.024
# ror_err = 0.00012
# # m = np.ones_like(centre, dtype=bool)

# phase_in = phase[m23]

# reloaded_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}

# relo = model.ReloadedModel(phase_in, **reloaded_kwargs)

# def _log_prior(theta):
    
    
# #     vsini, ell = theta
#     vsini, ell, roa_step, cosi_step = theta
#     b_step  = cosi_step / roa_step
    
#     if vsini < 0.0 or vsini > 20.0:
#         return -np.inf
#     elif ell < -180.0 or ell > 180.0:
#         return -np.inf
#     elif cosi_step < 0 or cosi_step > 0.1:
#         return -np.inf
#     elif roa_step < 0 or roa_step > 1:
#         return -np.inf
# #     elif ror_step < 0 or ror_step > 1:
# #         return -np.inf
#     elif b_step < 0 or b_step > 1: # really b_step < 1 + ror, but we know the planet transt is not grazing
#         return -np.inf
    
#     incl_step = np.rad2deg(np.arccos(cosi_step))
    
#     l = 0
#     l += -0.5 * ((incl_step - incl)**2 / incl_err**2)
#     l += -0.5 * ((roa_step - roa)**2 / roa_err**2)
# #     l += -0.5 * ((ror_step - ror)**2 / ror_err**2)
    
#     return l

# def _log_likelihood(data, model, error):
#         inv_sigma2 = 1/error**2
#         return -0.5 * np.sum((data - model)**2 * inv_sigma2 - np.log(inv_sigma2))

# def _log_probability(theta):

#     l = _log_prior(theta)
#     if not np.isfinite(l):
#         return -np.inf
    
# #     vsini, ell = theta
#     vsini, ell, roa_step, cosi_step = theta
    
#     incl_step = np.rad2deg(np.arccos(cosi_step))
#     b_step  = cosi_step / roa_step
    
#     dur23 = utils.get_23_transit_duration(period, roa_step, ror, b_step, np.deg2rad(incl_step))
# #     dur14 = utils.get_14_transit_duration(period, roa, ror, b, np.deg2rad(incl_step))
#     m = utils.get_transit_mask(phase_in, dur23/period)
# #     if np.count_nonzero(m) < len(phase_in):
# #         return -np.inf
    
#     if not np.any(m):
#         return -np.inf

#     reloaded_kwargs = {'r_1':roa_step, 'i_p':incl_step, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
#     relo = model.ReloadedModel(phase_in[m], **reloaded_kwargs)
#     mod = relo(vsini, ell, istar, alpha)
# #     print(mod)
# #     print(centre[m])
# #     print(len(centre[m]))
#     l += _log_likelihood(centre[m], mod, centre_err[m])
# #     l += _log_likelihood(centre, mod, centre_err)
# #     print(l)
#     if not np.isfinite(l):
#         print(l, vsini, ell, roa, b, np.any(~m))
#         return -np.inf
#     return l

# parameters = ['vsini', 'ell', 'roa', 'cosi']
# ndim = len(parameters)

# start = np.array([5, 0, roa, np.cos(np.deg2rad(incl))])
# error = np.array([2, 45, roa_err, 0.0001])

# init = []
# for i in range(walkers):
#     pos = start + error * np.random.randn(ndim)
#     while not np.isfinite(_log_prior(pos)):
#         pos = start + error * np.random.randn(ndim)

#     init.append(pos)

# if threads > 1:
#     os.environ["OMP_NUM_THREADS"] = "1"
#     with Pool(processes=threads) as pool:
#         sampler = emcee.EnsembleSampler(walkers, ndim,
#                                         _log_probability,
#                                         pool=pool)
# #                                         args=(phase, rv, rv_err))
#         sampler.run_mcmc(init, steps, progress=True)
# else:
#     sampler = emcee.EnsembleSampler(walkers, ndim,
#                                         _log_probability)
# #                                         args=(phase, rv, rv_err))
#     sampler.run_mcmc(init, steps, progress=True)

100%|██████████| 5000/5000 [07:16<00:00, 11.46it/s]


In [48]:
discard = int(0.5 * steps)
thin = int(np.mean(sampler.get_autocorr_time(discard=discard)))

stepsarr = np.arange(int((steps-discard)/thin))

fig, axes = plt.subplots(ndim+1,1, figsize=(10,2*ndim),
        gridspec_kw={"hspace":0.04})


vsini = np.sum(sampler.get_chain(discard=discard, thin=thin)**2, axis=-1)
ell = np.rad2deg(np.arctan2(*np.rollaxis(sampler.get_chain(discard=discard, thin=thin), 2, 0)))
posterior_3d = np.dstack((vsini, ell))


labels = ['logp', '$v\sin{i}$ (km/s)', '$\lambda$ (deg)']

for i in range(ndim+1):
    axes[i].set_xlim(0, stepsarr.max())
    for j in range(walkers):
        if i == 0:
            axes[i].plot(stepsarr, sampler.get_log_prob(discard=discard, thin=thin)[:,j], lw=0.5)
        else:
            axes[i].plot(stepsarr, posterior_3d[:,j,i-1], lw=0.5)
        axes[i].set_ylabel(labels[i])
        if i == ndim:
            axes[i].set_xlabel('steps')
        else:
            axes[i].tick_params(labelbottom=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
import corner

labels = ["$v\sin{i}$ (km/s)", "$\lambda$ (deg)"]

posterior_2d = posterior_3d.reshape((-1, posterior_3d.shape[-1]))

fig = corner.corner(
    posterior_2d,
             labels=labels, show_titles=True, title_fmt=".4f", bins=30)#,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# roa_prior = np.random.normal(roa, roa_err, sampler.get_chain(flat=True, discard=discard, thin=thin).shape[0])
# cosi_prior = np.cos(
#                     np.deg2rad(
#                             np.random.normal(
#                                     incl, incl_err, sampler.get_chain(flat=True, discard=discard, thin=thin).shape[0]
#                             )
#                     )
#             )

# b  = sampler.get_chain(flat=True, discard=discard, thin=thin)[:,-1] / sampler.get_chain(flat=True, discard=discard, thin=thin)[:,2]
# data = np.column_stack((sampler.get_chain(flat=True, discard=discard, thin=thin), b))
# print(data.shape)

# figsize=(6,6))

# fig.axes[10].hist(roa_prior, bins=30, histtype='step')
# fig.axes[10].set_xlim(0)
# fig.axes[12].hist(roa_prior, bins=30, histtype='step')
# fig.axes[12].set_xlim(0)
# fig.axes[-1].hist(cosi_prior, bins=30, histtype='step')

            
# axes[3].axhline(roa, c="#aaaaaa")
# axes[4].axhline(np.cos(np.deg2rad(incl)), c="#aaaaaa")

In [166]:
plt.figure()
plt.hist(b, histtype='step', bins=30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  4.,  17.,  31.,  70., 129., 162., 258., 278., 313., 374., 361.,
        366., 331., 334., 286., 208., 203.,  96.,   2.,   0.,   0.,  68.,
        465., 377., 342., 242., 222., 175., 162., 124.]),
 array([0.40757658, 0.42732029, 0.447064  , 0.46680771, 0.48655143,
        0.50629514, 0.52603885, 0.54578256, 0.56552627, 0.58526998,
        0.6050137 , 0.62475741, 0.64450112, 0.66424483, 0.68398854,
        0.70373226, 0.72347597, 0.74321968, 0.76296339, 0.7827071 ,
        0.80245081, 0.82219453, 0.84193824, 0.86168195, 0.88142566,
        0.90116937, 0.92091309, 0.9406568 , 0.96040051, 0.98014422,
        0.99988793]),
 <a list of 1 Patch objects>)

In [22]:


# index_ml = np.argmax(sampler.get_log_prob(discard=discard, flat=True, thin=thin))
# theta_ml = sampler.get_chain(discard=discard, flat=True, thin=thin)[index_ml,:]

# theta_ml2 = theta_ml.copy()
# # theta_ml2[0] = theta_ml[0]**2 + theta_ml[1]**2
# # theta_ml2[1] = np.rad2deg(np.arctan2(theta_ml[0], theta_ml[1]))
# # theta_ml2[3] = np.rad2deg(np.arccos(theta_ml[3]))
# theta_ml2[-1] = np.rad2deg(np.arccos(theta_ml[-1]))
# print(theta_ml2)

# phase_f = np.linspace(phase[m23].min(), phase[m23].max(), 100)
# relo = model.ReloadedModel(phase[m23], r_1=theta_ml2[2], i_p=theta_ml2[3])
# mod = relo(theta_ml2[0], theta_ml2[1], istar, alpha, phase=phase[m23])
# mu_avg = relo.mu_avg
# mod_f = relo(theta_ml2[0], theta_ml2[1], istar, alpha, phase=phase_f)

# res = centre - mod
# mod0 = np.mean(centre)
# res0 = centre - mod0
# print('chisq reduced = {0:.4f}'.format(np.sum((res / centre_err)**2) / (len(centre) - 2)))
# print('chisq reduced H0 = {0:.4f}'.format(np.sum((res0 / centre_err)**2) / (len(centre) - 2)))
# samples = []
# for i in np.random.randint(0,sampler.get_chain(flat=True, discard=discard, thin=thin).shape[0],100):
#     _roa = sampler.get_chain(flat=True, discard=discard, thin=thin)[i,2]
#     _cosi = sampler.get_chain(flat=True, discard=discard, thin=thin)[i,3]
#     _incl = np.rad2deg(np.arccos(_cosi))
#     _b  = _cosi / _roa
# #     reloaded_kwargs_tmp = {'r_1':r_1_mean, 'i_p':incl_mean, 'r_p':k, 'ld':'quad', 'ldc':[0.28, 0.27], 'Nxy':51}
#     reloaded_kwargs_tmp = {'r_1':_roa,
#                            'i_p':_incl,
#                            'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
#     dur23 = utils.get_23_transit_duration(period, _roa, ror, _b, np.deg2rad(_incl))
# #     dur14 = utils.get_14_transit_duration(period, roa, ror, b, np.deg2rad(incl_step))
#     m = utils.get_transit_mask(phase_in, dur23/period)
#     print(m)
#     relo = model.ReloadedModel(phase_in[m], **reloaded_kwargs_tmp)
#     _mod = relo(sampler.get_chain(flat=True, discard=discard, thin=thin)[i,0],
#                         sampler.get_chain(flat=True, discard=discard, thin=thin)[i,1],
#                         90, 0, phase=phase_f)
# #     print(_mod)
#     samples.append(_mod)
# samples = np.atleast_2d(samples)

# # samples = np.atleast_2d([relo(fc_convert[i,0], fc_convert[i,1], 90, 0, phase=phase_f) for i in np.random.randint(0,fc.shape[0],100)])
# # samples = np.atleast_2d([relo(fc_convert[i,0], fc_convert[i,1], 90, 0, phase=phase_f, r_1=fc_convert[i,2], i_p=fc_convert[i,3]) for i in np.random.randint(0,fc.shape[0],100)])
# # fig = utils.plot_fit(phase[m], rv[m], np.sqrt(rv_err[m]**2 + theta_ml[-1]**2), res, phase_f, mod_f, period=P)#, z=mu_avg)
# fig = utils.plot_fit(phase[m23], centre, centre_err, res, phase_f, mod_f, period=period, samples=samples, ylim=[-4,4])#, z=mu_avg)
# # fig.axes[0].plot(phase_f*P*24, relo(3.6, 0., 90., 0., phase=phase_f))
# # fig.axes[0].plot(phase_f*24, relo(3.6, 0., 90., 0., phase=phase_f))
# # fig.axes[0].axvline(-0.5*dur14*24, c='#aaaaaa')
# # fig.axes[0].axvline(-0.5*dur23*24, c='#aaaaaa', ls='dashed')
# # fig.axes[0].axvline(0.5*dur23*24, c='#aaaaaa', ls='dashed')
# # fig.axes[0].axvline(0.5*dur14*24, c='#aaaaaa')
# # fig.axes[1].set_ylim(-3, 3)
# print(istar, alpha)

In [58]:
index_ml = np.argmax(sampler.get_log_prob(discard=discard, thin=thin, flat=True))

# theta_ml = [x[index_ml] for x in posterior_2d.T]
theta_ml = posterior_2d[index_ml]
print(theta_ml)


phase_f = np.linspace(-0.5 * dur14/period, 0.5 * dur14/period, 200)
# phase_f = np.linspace(-0.01, 0.01, 200)
# phase_f = np.linspace(phase[0], phase[-1], 100)

# reloaded_kwargs = {'r_1':theta_ml[], 'i_p':theta_ml[3], 'r_p':k, 'ld':'quad', 'ldc':[0.28,  0.27], 'Nxy':51}
#                   'oversample':3, 'dp':900/60/60/24/P}

reloaded_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
relo = model.ReloadedModel(phase_in, **reloaded_kwargs)

# ymod1 = relo(theta_ml[0], theta_ml[1], istar, alpha, phase=phase1[m1])
# ymod2  = relo(theta_ml[0], theta_ml[1], istar, alpha, phase=phase2[m2])
mod = relo(theta_ml[0], theta_ml[1], istar, alpha)#, phase=phase_in)




# mu_avg = relo.mu_avg
# mod_f = relo(theta_ml[0], theta_ml[1], istar, alpha, phase=phase_f)
mod_f = relo(*theta_ml, istar, alpha, phase=phase_f)

# mod0_f = relo(theta_ml[0], 0, istar, alpha, phase=phase_f)
# mod90_f = relo(theta_ml[0], -90, istar, alpha, phase=phase_f)
# mod180_f = relo(theta_ml[0], 180, istar, alpha, phase=phase_f)


# print(6.2 * np.sin(np.deg2rad(120)) * (1 - 0.5*np.sin(np.deg2rad(45))**2))

# omc = centre - mod
# omc1 = rv1[m1] - ymod1
# omc2 = rv2[m2] - ymod2

    
# samples = np.atleast_2d([relo(chain3[i,0], chain3[i,1], istar, alpha,
#                                   r_1=r_1_mean, i_p=incl_mean, phase=phase_f)
#                              for i in np.random.randint(0,vsini.shape[0],100)])

samples = np.atleast_2d([relo(*posterior_2d[i], istar, alpha, phase=phase_f)
                         for i in np.random.randint(0,posterior_2d.shape[0],100)])
# mod_f = np.median(samples, axis=0)
# print(what)

# reloaded_kwargs = {'r_1':r_1, 'i_p':incl, 'r_p':k, 'ld':'power2', 'ldc':[c, alpha], 'Nxy':51}

# relo = model.ReloadedModel(phase[converged], **reloaded_kwargs)
# ynormmark = relo.y_norm_mark(np.deg2rad(45), np.deg2rad(100))[0,25,25]
# latitudes = [np.rad2deg(np.arcsin(x[25,25])) for x in relo.y_norm_mark(np.deg2rad(45), np.deg2rad(90))]
# print(latitudes)
# print(ynormmark)
# print(6.2 * np.sin(np.deg2rad(100)) * (1 - 0.5 * ynormmark**2))
# print(samples)

# period = P

# xf = phase_f
# x = phase[m]
# y = rv[m]
# yerr = rv_err[m]

# x = [phase1[m1], phase2[m2]]
# y = [rv_fit1[m1], rv_fit2[m2]]
# yerr = [rv_err_fit1[m1], rv_err_fit2[m2]]


# if isinstance(x, np.ndarray):
#     x = [x]
# if isinstance(y, np.ndarray):
#     y = [y]
# if isinstance(yerr, np.ndarray):
#     yerr = [yerr]
# if isinstance(omc, np.ndarray):
#     omc = [omc]
# if isinstance(omc_d, np.ndarray):
#     omc_d = [omc_d]


# n = len(x)


# if samples is not None:
#     samples = np.atleast_2d(samples)

    
plt.style.use('default')
# figsize = [None, 3]
# fs = [5, 4.42]
# figsize=None
# if figsize is None:
#     fs = figsize
# elif None in figsize:

#     fig    = plt.figure()
#     w, h   = figsize
#     _w, _h = fig.get_size_inches()

#     w = _w if w is None else w
#     h = _h if h is None else h

#     fs = (w, h)

ncols = 1
gridspec_kw = {
              'height_ratios':[3,1], 
              'hspace':0.03,
              'wspace':0.02
              }



# gs = GridSpec(2, ncols, **gridspec_kw)
# fig = plt.figure(figsize=fs) # if twocol

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridspec_kw)
# fig = plt.subplots(2, 1, gridspec_kw=gridspec_kw, squeeze=True)
print(ax1)
print(fig)
# fig = plt.figure()

# ax1 = fig.add_subplot(gs[0,0])
# ax2 = fig.add_subplot(gs[1,0], sharex=ax1)
# ax3 = fig.add_subplot(gs[0,1], sharey=ax1)
# ax4 = fig.add_subplot(gs[1,1], sharex=ax3, sharey=ax2)


ax1.set_ylabel('stellar surface velocity (km/s)')
ax1.tick_params(axis='x', which='both', labelbottom=False)
# ax3.tick_params(axis='both', which='both', labelbottom=False, labelleft=False)
# ax4.tick_params(axis='y', which='both', labelleft=False)

c1, c2, c3, c4 = 0.5 / period * np.array([-dur14, -dur23, dur23, dur14])


# 14.608559
ax1.axvline(c1, c='#aaaaaa', lw=1, ls='solid')
ax1.axvline(c2, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c3, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c4, c='#aaaaaa', lw=1, ls='solid')

# ax2.axvline(-W14*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax2.axvline(-W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax2.axvline(W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax2.axvline(W14*24/2, c='#aaaaaa', lw=1, ls='dotted')

# ax3.axvline(-W14*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax3.axvline(-W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax3.axvline(W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax3.axvline(W14*24/2, c='#aaaaaa', lw=1, ls='dotted')

# ax4.axvline(-W14*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax4.axvline(-W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax4.axvline(W23*24/2, c='#aaaaaa', lw=1, ls='dotted')
# ax4.axvline(W14*24/2, c='#aaaaaa', lw=1, ls='dotted')

# if ylim is None:
offset = np.median(centre_err)
ylim = (mod.min()-offset, mod.max()+offset)
ax1.set_ylim(ylim)


# offset = np.median(yerr)
# ylim = (ymod.min()-offset, ymod.max()+offset)
# ax1.set_ylim(ylim)
# ax1.set_xlim(xf[0]*P*24, xf[-1]*P*24)


ax2.tick_params(axis='x', which='both', top=True)
# ax4.tick_params(axis='x', which='both', top=True)
xlabel = 'phase'
# if period is not None:
#     xlabel += ' (hours)'
    
ax2.set_ylabel('O - C')

ax2.set_xlabel(xlabel)
# ax4.set_xlabel(xlabel)

# residuals
# ax2.set_ylim(-5*np.median(centre_err), 5*np.median(centre_err))

markers = ['.', 's']
# 
colors = ['k', 'C1']



kwargs = {'capsize':0, 'fmt':'none', 'color':colors[0],
            'alpha':1.0, 'elinewidth':1.0}
ax1.errorbar(phase_in, centre, yerr=centre_err, **kwargs)
ax2.errorbar(phase_in, centre - mod, yerr=centre_err,
        **kwargs)

# kwargs = {'capsize':0, 'fmt':'none', 'color':colors[0],
#             'alpha':1.0, 'elinewidth':1.0}
# ax1.errorbar(phase1[m1]*period*24, rv1[m1], yerr=rv_err1[m1], **kwargs)
# ax2.errorbar(phase1[m1]*period*24, omc1, yerr=rv_err1[m1],
#         **kwargs)


kwargs = {'fmt':markers[0], 'color':colors[0], 'ms':10}
ax1.errorbar(phase_in, centre, **kwargs)
ax2.errorbar(phase_in, centre - mod, **kwargs)


# kwargs = {'fmt':markers[0], 'color':'white', 'ms':7.8, 'mew':0}
# ax1.errorbar(phase1[m1]*period*24, rv1[m1], **kwargs)
# ax2.errorbar(phase1[m1]*period*24, omc1, **kwargs)




# kwargs = {'capsize':0, 'fmt':'none', 'color':colors[1],
#             'alpha':1.0, 'elinewidth':1.0, 'zorder':-100}
# ax1.errorbar(phase2[m2]*period*24, rv2[m2], yerr=rv_err2[m2], **kwargs)
# ax2.errorbar(phase2[m2]*period*24, omc2, yerr=rv_err2[m2],
#         **kwargs)


# kwargs = {'fmt':markers[1], 'color':colors[1], 'ms':4.5, 'zorder':-100}
# ax1.errorbar(phase2[m2]*period*24, rv2[m2], label='B', **kwargs)
# ax2.errorbar(phase2[m2]*period*24, omc2, **kwargs)
# ax1.legend(loc='upper left')

# kwargs = {'fmt':markers[1], 'color':'white', 'ms':3.5, 'mew':0, 'zorder':-100}
# ax1.errorbar(phase2[m2]*period*24, rv2[m2], **kwargs)
# ax2.errorbar(phase2[m2]*period*24, omc2, **kwargs)
    
    

ax2.axhline(0, c="#aaaaaa", lw=1)

ax1.set_xlim(c1, c4)
ax2.set_xlim(c1, c4)

ax1.set_ylim(-3, 3)
# ax2.set_ylim(-0.2, 0.2)


# ax1.plot(phase_f, mod_f, color='#aaaaaa', lw=0.5, zorder=-150)

cmap = plt.get_cmap('Blues')

percs = np.linspace(51, 99, 100)

colors = (percs - np.min(percs)) / (np.max(percs) - np.min(percs))

for i, p in enumerate(percs[::-1]):
    upper = np.percentile(samples, p, axis=0)
    lower = np.percentile(samples, 100-p, axis=0)
    color_val = colors[i]
    ax1.fill_between(phase_f, upper, lower, color=cmap(color_val), alpha=0.8, zorder=-200)#, **fill_kwargs)
    

[ 3.26532976 -0.44217832]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AxesSubplot(0.125,0.311034;0.775x0.568966)
Figure(640x480)


In [90]:
texp * 24 * 60 * 60

array([900.0011, 900.0017, 900.0012, 600.0021, 600.0015, 600.0028,
       600.0024, 600.0019, 600.0014, 600.0002, 600.0015, 600.0009,
       600.0008, 600.0015, 599.9999, 600.0007, 600.0014, 600.0007,
       599.9998, 600.0005])

In [105]:
from multiprocessing import Pool
import emcee
sys.path.append("/Users/vxh710/PhD/software/elle")
import model


threads = 2
walkers = 400
steps = 10000

bounds = ((-5, 5), (-5, 5), (-1, 1), (0, 1))

reloaded_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
#                   'oversample':5, 'dp':600/60/60/24/period}

relo = model.ReloadedModel(phase_in, **reloaded_kwargs)

def _log_prior(theta):
        
    vs, vc, cosi, alpha = theta
    
    if vs < bounds[0][0] or vs > bounds[0][1]:
        return -np.inf
    elif vc < bounds[1][0] or vc > bounds[1][1]:
        return -np.inf
    elif cosi < bounds[2][0] or cosi > bounds[2][1]:
        return -np.inf
    elif alpha < bounds[3][0] or alpha > bounds[3][1]:
        return -np.inf

    return 0

def _log_likelihood(data, model, error):
        inv_sigma2 = 1/error**2
        return -0.5 * np.sum((data - model)**2 * inv_sigma2 - np.log(inv_sigma2))

def _log_probability(theta):

    # calculate prior and check the new parameters are within bounds
    l = _log_prior(theta)
    
    if not np.isfinite(l):
        return -np.inf
    
    # calculate vsini and lambda from the free parameters
    vs, vc, cosi, alpha = theta # vs = sqrt(vsini) * np.sin(lambda); vc = sqrt(vsini) * np.cos(lambda)
    veq = vs**2 + vc**2  
    ell = np.rad2deg(np.arctan2(vs, vc))
    istar = np.rad2deg(np.arccos(cosi))

    mod = relo(veq, ell, istar, alpha) # calculate surface RV model

    l += _log_likelihood(centre, mod, centre_err)
    
    return l

parameters = ['vs', 'vc', 'cosi', 'alpha']
ndim = len(parameters)

init = []
for i in range(walkers):
    pos = [np.random.uniform(*bound) for bound in bounds]
    
    # check that the initial positions are within prior bounds to be safe
    while not np.isfinite(_log_prior(pos)):
        pos = [np.random.uniform(*bound) for bound in bounds]

    init.append(pos)

    
if threads > 1:
    os.environ["OMP_NUM_THREADS"] = "1"
    with Pool(processes=threads) as pool:
        sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability,
                                        pool=pool)
#                                         args=(phase, rv, rv_err))
        sampler.run_mcmc(init, steps, progress=True)
else:
    sampler = emcee.EnsembleSampler(walkers, ndim,
                                        _log_probability)
#                                         args=(phase, rv, rv_err))
    sampler.run_mcmc(init, steps, progress=True)

100%|██████████| 10000/10000 [18:38<00:00,  8.94it/s]


In [108]:
sampler.get_autocorr_time(discard=5000)

AutocorrError: The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 100;
tau: [145.33193711 318.31650407 341.36206497 257.97477934]

In [109]:
# discard = int(0.5 * steps)
# thin = int(np.mean(sampler.get_autocorr_time(discard=discard)))

discard = 5000
thin = 300

stepsarr = np.arange(int((steps-discard)/thin))

fig, axes = plt.subplots(ndim+1,1, figsize=(10,2*ndim),
        gridspec_kw={"hspace":0.04})


vsini = np.sum(sampler.get_chain(discard=discard, thin=thin)[:,:,:2]**2, axis=-1)
ell = np.rad2deg(np.arctan2(*np.rollaxis(sampler.get_chain(discard=discard, thin=thin)[:,:,:2], 2, 0)))
istar = np.rad2deg(np.arccos(sampler.get_chain(discard=discard, thin=thin)[:,:,2]))
posterior_3d = np.dstack((vsini, ell, istar, sampler.get_chain(discard=discard, thin=thin)[:,:,3]))

labels = ['logp', r'$v_\mathrm{eq}$ (km/s)', r'$\lambda$ (deg)', r'$i_\star$ (deg)', r'$\alpha$']

for i in range(ndim+1):
    axes[i].set_xlim(0, stepsarr.max())
    for j in range(walkers):
        if i == 0:
            axes[i].plot(stepsarr, sampler.get_log_prob(discard=discard, thin=thin)[:,j], lw=0.5)
        else:
            axes[i].plot(stepsarr, posterior_3d[:,j,i-1], lw=0.5)
        axes[i].set_ylabel(labels[i])
        if i == ndim:
            axes[i].set_xlabel('steps')
        else:
            axes[i].tick_params(labelbottom=False)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
import corner

# labels = ["$v\sin{i}$ (km/s)", "$\lambda$ (deg)"]

posterior_2d = posterior_3d.reshape((-1, posterior_3d.shape[-1]))

fig = corner.corner(
    posterior_2d,
             labels=labels[1:], show_titles=True, title_fmt=".4f", bins=30)#,

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
index_ml = np.argmax(sampler.get_log_prob(discard=discard, thin=thin, flat=True))

theta_ml = posterior_2d[index_ml]
print(theta_ml)


phase_f = np.linspace(-0.5 * dur14/period, 0.5 * dur14/period, 200)


reloaded_kwargs = {'r_1':roa, 'i_p':incl, 'r_p':ror, 'ld':'quad', 'ldc':u, 'Nxy':51}
relo = model.ReloadedModel(phase_in, **reloaded_kwargs)

mod = relo(theta_ml[0], theta_ml[1], theta_ml[2], theta_ml[3])

mod_f = relo(*theta_ml, phase=phase_f)


samples = np.atleast_2d([relo(*posterior_2d[i], phase=phase_f)
                         for i in np.random.randint(0,posterior_2d.shape[0],100)])


ncols = 1
gridspec_kw = {
              'height_ratios':[3,1], 
              'hspace':0.03,
              'wspace':0.02
              }

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridspec_kw)

ax1.set_ylabel('stellar surface velocity (km/s)')
ax1.tick_params(axis='x', which='both', labelbottom=False)


c1, c2, c3, c4 = 0.5 / period * np.array([-dur14, -dur23, dur23, dur14])

ax1.axvline(c1, c='#aaaaaa', lw=1, ls='solid')
ax1.axvline(c2, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c3, c='#aaaaaa', lw=1, ls='dotted')
ax1.axvline(c4, c='#aaaaaa', lw=1, ls='solid')

offset = np.median(centre_err)
ylim = (mod.min()-offset, mod.max()+offset)
ax1.set_ylim(ylim)



ax2.tick_params(axis='x', which='both', top=True)
ax2.set_ylabel('O - C')

ax2.set_xlabel('phase')


markers = ['.', 's']
# 
colors = ['k', 'C1']



kwargs = {'capsize':0, 'fmt':'none', 'color':colors[0],
            'alpha':1.0, 'elinewidth':1.0}
ax1.errorbar(phase_in, centre, yerr=centre_err, **kwargs)
ax2.errorbar(phase_in, centre - mod, yerr=centre_err,
        **kwargs)



kwargs = {'fmt':markers[0], 'color':colors[0], 'ms':10}
ax1.errorbar(phase_in, centre, **kwargs)
ax2.errorbar(phase_in, centre - mod, **kwargs)
    
    

ax2.axhline(0, c="#aaaaaa", lw=1)

ax1.set_xlim(c1, c4)
ax2.set_xlim(c1, c4)

ax1.set_ylim(-4, 4)

cmap = plt.get_cmap('Blues')

percs = np.linspace(51, 99, 100)

colors = (percs - np.min(percs)) / (np.max(percs) - np.min(percs))

for i, p in enumerate(percs[::-1]):
    upper = np.percentile(samples, p, axis=0)
    lower = np.percentile(samples, 100-p, axis=0)
    color_val = colors[i]
    ax1.fill_between(phase_f, upper, lower, color=cmap(color_val), alpha=0.8, zorder=-200)#, **fill_kwargs)
    

[ 5.12608022 -0.19886369 97.85334224  0.69757022]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …